<table>
    <tr>
        <td><img src="./img/Macc.png" width="auto"/></td>
        <td>
            <table><tr>
            <h1 style="color:blue;text-align:center">Lógica para Ciencias de la Computación</h1></td>
            </tr></table>   
        <td>&nbsp;</td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Grupo 6</p></tp>
            <tp><p style="font-size:150%;text-align:center">Representación del problema</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

# Objetivo <a class="anchor" id="inicio"></a>

Implementar en Python la representación del problema que nuestro grupo desea abordar, esto de acuerdo a lo establecido en la primera entrega y a las modificaciones de la misma.

# Secciones

1. [Problema a resolver.](#prob)
2. [Descriptores.](#des)
3. [Implementación de restricciones.](#imp)

# Problema a resolver <a class="anchor" id="prob"></a>

([Volver al inicio](#inicio))

Trabajaremos sobre el siguiente problema: Se busca ubicar minimo 3 $X$ en cada fila y columna sobre un tablero de de tamaño 7x7 de manera que no queden tres $X$ seguidas en ninguna dirección.

Un ejemplo de una posible solución al problema sería la siguiente

![ejemplo](img/tabla.png)

Este problema cuenta con tres restricciones en total:

1. Solo se permite una $X$ por casilla.
2. No deben haber tres $X$ seguidas en ninguna dirección (horizontal, vertical o diagonales).
3. Deben haber mínimo tres $X$ en cada fila y columna.


# Descriptores <a class="anchor" id="des"></a>

([Volver al inicio](#inicio))

Para empezar, deben definirse las letras proposicionales, las cuales definen la posición de cada $X$ en el tablero:

$XenC_{x,y}$ es verdadera sii hay una $X$ ubicada en la casilla $(x,y)$, siendo $x$ las filas y $y$ las columnas

Para hacer la implementación en python, se usa la clase `Descriptor`, que se encuentra en la librería `Logica`.  Esta clase permite codificar un átomo, representándolo como un solo caracter (es decir, una cadena de longitud 1).

Importamos la librería:

In [15]:
from Logica import *

Ahora creamos un descriptor de dos argumentos, siendo ambos para representar la casilla $(x,y)$:

In [16]:
Nx = 7
Ny = 7
X = list(range(Nx))
Y = list(range(Ny))
XenC = Descriptor([Nx, Ny])

### Codificando en un solo caracter

Mediante el método P() podemos crear las codificaciones. A continuación presentaremos cada uno de los caracteres que codifican los cruces de información de que una $X$ se encuentre en una casilla $(x,y)$ (donde $x,y\in\{0, 1, 2, 3, 4, 5, 6\}$):

In [17]:
print("Cantidad de átomos XenC:", XenC.rango[1] - XenC.rango[0])
print("Caracteres correspondientes a los átomos XenC:\n")
for x in range(Nx):
    for y in range(Ny):
        atomo = XenC.P([x,y])
        print(f"Hay una X en la casilla ({X[x]},{Y[y]}) es el átomo {atomo}")
    print("")

Cantidad de átomos XenC: 49
Caracteres correspondientes a los átomos XenC:

Hay una X en la casilla (0,0) es el átomo Ā
Hay una X en la casilla (0,1) es el átomo ć
Hay una X en la casilla (0,2) es el átomo Ď
Hay una X en la casilla (0,3) es el átomo ĕ
Hay una X en la casilla (0,4) es el átomo Ĝ
Hay una X en la casilla (0,5) es el átomo ģ
Hay una X en la casilla (0,6) es el átomo Ī

Hay una X en la casilla (1,0) es el átomo ā
Hay una X en la casilla (1,1) es el átomo Ĉ
Hay una X en la casilla (1,2) es el átomo ď
Hay una X en la casilla (1,3) es el átomo Ė
Hay una X en la casilla (1,4) es el átomo ĝ
Hay una X en la casilla (1,5) es el átomo Ĥ
Hay una X en la casilla (1,6) es el átomo ī

Hay una X en la casilla (2,0) es el átomo Ă
Hay una X en la casilla (2,1) es el átomo ĉ
Hay una X en la casilla (2,2) es el átomo Đ
Hay una X en la casilla (2,3) es el átomo ė
Hay una X en la casilla (2,4) es el átomo Ğ
Hay una X en la casilla (2,5) es el átomo ĥ
Hay una X en la casilla (2,6) es el átomo 

### Decodificando el caracter

Ahora podemos incluir un método para visualizar más fácilmente la información que porta cada letra proposicional. Esto es, en nuestro ejemplo en cuestión, nuestro objeto `XenC` representa el cruce de información de que una $X$ encuentra en un lugar. Entonces, al decodificar una letra proposicional, queremos que nos presente claramente esta información y no un caracter inpronunciable (aunque es precisamente este caracter el que usará la máquina). 

Para ello podemos usar el siguiente método `escribir`: 

In [18]:
def escribir(self, literal):
    if '-' in literal:
        atomo = literal[1:]
        neg = 'No h'
    else:
        atomo = literal
        neg = 'H'
    x, y= self.inv(atomo)
    return f"{neg}ay una X en la casilla ({X[x]},{Y[y]})"
    
from types import MethodType

XenC.escribir = MethodType(escribir, XenC)

---

# Implementación de restricciones <a class="anchor" id="imp"></a>

([Volver al inicio](#inicio))

Ahora es necesario crear las reglas que limitarán los valores de verdad para las letras proposicionales. En nuestro problema tenemos tres restricciones:

1. Solo se permite una $X$ por casilla.
2. No deben haber tres $X$ seguidas en ninguna dirección (horizontal, vertical o diagonales).
3. Deben haber mínimo tres $X$ en cada fila y columna.

---

## Restricción 1

Comencemos por considerar la restricción 1, que dice que sólo se permite una $X$ casilla. Note que esta restricción ya se cumple con la implementación del descriptor:

$$\large XenC_{x,y}$$

Ya que el mismo se asegura de que en cierta posición solo se encuentre una sola $X$

---

## Restricción 2

Ahora consideremos la restricción 2, que dice no puede haber 3 $X$ seguidas en ninguna dirección.

En este caso, se deben revisar 4 direcciones, siendo horizontal, vertical y ambas diagonales. Para comenzar,  se define cada dirección por separado:

### Horizontal ($\rightarrow$)

La fórmula para representar que no pueden haber 3 $X$ seguidas en horizontal es la siguiente:

$$\large \bigwedge_{x \in Filas\atop { y \in\{0, 4\}}}\biggl(XenC_{x, y}\wedge XenC_{x, y+1}\to\neg\left(XenC_{x, y+2}\right)\biggr)$$

Esta fórmula se implementa en Python de la siguiente manera:

In [19]:
formulaH = ''
inicial = True
for x in X:
    for y in range(5):
        if inicial:
            formulaH = '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x,y+1]) + ')' + '>-' + XenC.P([x,y+2]) + ')'
            inicial = False
        else:
            formulaH = "(" + formulaH + "Y" + '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x,y+1]) + ')' + '>-' + XenC.P([x,y+2]) + '))'

print("--------------------------------------------------FORMULA H CODIFICADA---------------------------------------------------------")
print("")
print(formulaH)
print("")
print("--------------------------------------------------FORMULA H DESCODIFICADA------------------------------------------------------")
print("")
print(visualizar_formula(formulaH, XenC))

--------------------------------------------------FORMULA H CODIFICADA---------------------------------------------------------

((((((((((((((((((((((((((((((((((((ĀYć)>-Ď)Y((ćYĎ)>-ĕ))Y((ĎYĕ)>-Ĝ))Y((ĕYĜ)>-ģ))Y((ĜYģ)>-Ī))Y((āYĈ)>-ď))Y((ĈYď)>-Ė))Y((ďYĖ)>-ĝ))Y((ĖYĝ)>-Ĥ))Y((ĝYĤ)>-ī))Y((ĂYĉ)>-Đ))Y((ĉYĐ)>-ė))Y((ĐYė)>-Ğ))Y((ėYĞ)>-ĥ))Y((ĞYĥ)>-Ĭ))Y((ăYĊ)>-đ))Y((ĊYđ)>-Ę))Y((đYĘ)>-ğ))Y((ĘYğ)>-Ħ))Y((ğYĦ)>-ĭ))Y((ĄYċ)>-Ē))Y((ċYĒ)>-ę))Y((ĒYę)>-Ġ))Y((ęYĠ)>-ħ))Y((ĠYħ)>-Į))Y((ąYČ)>-ē))Y((ČYē)>-Ě))Y((ēYĚ)>-ġ))Y((ĚYġ)>-Ĩ))Y((ġYĨ)>-į))Y((ĆYč)>-Ĕ))Y((čYĔ)>-ě))Y((ĔYě)>-Ģ))Y((ěYĢ)>-ĩ))Y((ĢYĩ)>-İ))

--------------------------------------------------FORMULA H DESCODIFICADA------------------------------------------------------

((((((((((((((((((((((((((((((((((((Hay una X en la casilla (0,0) Y Hay una X en la casilla (0,1)) >  no Hay una X en la casilla (0,2)) Y ((Hay una X en la casilla (0,1) Y Hay una X en la casilla (0,2)) >  no Hay una X en la casilla (0,3))) Y ((Hay una X en la casilla (0,2) Y Hay una X e

---

### Vertical ($\downarrow$)

La fórmula para representar que no pueden haber 3 $X$ seguidas en vertical es la siguiente:

$$\large \bigwedge_{x \in\{0, 4\}\atop { y \in Columnas}}\biggl(XenC_{x, y}\wedge XenC_{x+1, y}\to\neg\left(XenC_{x+2, y}\right)\biggr)$$

Esta fórmula se implementa en Python de la siguiente manera:


In [20]:
formulaV = ''
inicial = True
for y in Y:
    for x in range(5):
        if inicial:
            formulaV = '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x+1,y]) + ')' + '>-' + XenC.P([x+2,y]) + ')'
            inicial = False
        else:
            formulaV = "(" + formulaV + "Y" + '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x+1,y]) + ')' + '>-' + XenC.P([x+2,y]) + '))'

print("--------------------------------------------------FORMULA V CODIFICADA---------------------------------------------------------")
print("")
print(formulaV)
print("")
print("--------------------------------------------------FORMULA V DESCODIFICADA------------------------------------------------------")
print("")
print(visualizar_formula(formulaV, XenC))

--------------------------------------------------FORMULA V CODIFICADA---------------------------------------------------------

((((((((((((((((((((((((((((((((((((ĀYā)>-Ă)Y((āYĂ)>-ă))Y((ĂYă)>-Ą))Y((ăYĄ)>-ą))Y((ĄYą)>-Ć))Y((ćYĈ)>-ĉ))Y((ĈYĉ)>-Ċ))Y((ĉYĊ)>-ċ))Y((ĊYċ)>-Č))Y((ċYČ)>-č))Y((ĎYď)>-Đ))Y((ďYĐ)>-đ))Y((ĐYđ)>-Ē))Y((đYĒ)>-ē))Y((ĒYē)>-Ĕ))Y((ĕYĖ)>-ė))Y((ĖYė)>-Ę))Y((ėYĘ)>-ę))Y((ĘYę)>-Ě))Y((ęYĚ)>-ě))Y((ĜYĝ)>-Ğ))Y((ĝYĞ)>-ğ))Y((ĞYğ)>-Ġ))Y((ğYĠ)>-ġ))Y((ĠYġ)>-Ģ))Y((ģYĤ)>-ĥ))Y((ĤYĥ)>-Ħ))Y((ĥYĦ)>-ħ))Y((ĦYħ)>-Ĩ))Y((ħYĨ)>-ĩ))Y((ĪYī)>-Ĭ))Y((īYĬ)>-ĭ))Y((ĬYĭ)>-Į))Y((ĭYĮ)>-į))Y((ĮYį)>-İ))

--------------------------------------------------FORMULA V DESCODIFICADA------------------------------------------------------

((((((((((((((((((((((((((((((((((((Hay una X en la casilla (0,0) Y Hay una X en la casilla (1,0)) >  no Hay una X en la casilla (2,0)) Y ((Hay una X en la casilla (1,0) Y Hay una X en la casilla (2,0)) >  no Hay una X en la casilla (3,0))) Y ((Hay una X en la casilla (2,0) Y Hay una X e

---

### Diagonal Derecha ($\searrow$)

La fórmula para representar que no pueden haber 3 $X$ seguidas en diagonal derecha ($\searrow$) es la siguiente:

$$\large \bigwedge_{x \in\{0, 4\}\atop { y \in\{0, 4\}}}\biggl(XenC_{x, y}\wedge XenC_{x+1, y+1}\to\neg\left(XenC_{x+2, y+2}\right)\biggr)$$

Esta fórmula se implementa en Python de la siguiente manera:


In [21]:
formulaDD = ''
inicial = True
for x in range(5):
    for y in range(5):
        if inicial:
            formulaDD = '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x+1,y+1]) + ')' + '>-' + XenC.P([x+2,y+2]) + ')'
            inicial = False
        else:
            formulaDD = "(" + formulaDD + "Y" + '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x+1,y+1]) + ')' + '>-' + XenC.P([x+2,y+2]) + '))'

print("--------------------------------------------------FORMULA DD CODIFICADA--------------------------------------------------------")
print("")
print(formulaDD)
print("")
print("--------------------------------------------------FORMULA DD DESCODIFICADA-----------------------------------------------------")
print("")
print(visualizar_formula(formulaDD, XenC))

--------------------------------------------------FORMULA DD CODIFICADA--------------------------------------------------------

((((((((((((((((((((((((((ĀYĈ)>-Đ)Y((ćYď)>-ė))Y((ĎYĖ)>-Ğ))Y((ĕYĝ)>-ĥ))Y((ĜYĤ)>-Ĭ))Y((āYĉ)>-đ))Y((ĈYĐ)>-Ę))Y((ďYė)>-ğ))Y((ĖYĞ)>-Ħ))Y((ĝYĥ)>-ĭ))Y((ĂYĊ)>-Ē))Y((ĉYđ)>-ę))Y((ĐYĘ)>-Ġ))Y((ėYğ)>-ħ))Y((ĞYĦ)>-Į))Y((ăYċ)>-ē))Y((ĊYĒ)>-Ě))Y((đYę)>-ġ))Y((ĘYĠ)>-Ĩ))Y((ğYħ)>-į))Y((ĄYČ)>-Ĕ))Y((ċYē)>-ě))Y((ĒYĚ)>-Ģ))Y((ęYġ)>-ĩ))Y((ĠYĨ)>-İ))

--------------------------------------------------FORMULA DD DESCODIFICADA-----------------------------------------------------

((((((((((((((((((((((((((Hay una X en la casilla (0,0) Y Hay una X en la casilla (1,1)) >  no Hay una X en la casilla (2,2)) Y ((Hay una X en la casilla (0,1) Y Hay una X en la casilla (1,2)) >  no Hay una X en la casilla (2,3))) Y ((Hay una X en la casilla (0,2) Y Hay una X en la casilla (1,3)) >  no Hay una X en la casilla (2,4))) Y ((Hay una X en la casilla (0,3) Y Hay una X en la casilla (1,4)) >  no Hay una X

---

### Diagonal Izquierda ($\swarrow$)

La fórmula para representar que no pueden haber 3 $X$ seguidas en diagonal izquierda ($\swarrow$) es la siguiente:

$$\large \bigwedge_{x \in\{0, 4\}\atop { y \in\{2, 6\}}}\biggl(XenC_{x, y}\wedge XenC_{x+1, y-1}\to\neg\left(XenC_{x+2, y-2}\right)\biggr)$$

Esta fórmula se implementa en Python de la siguiente manera:


In [22]:
formulaDI = ''
inicial = True
for x in range(5):
    for y in range(2,7):
        if inicial:
            formulaDI = '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x+1,y-1]) + ')' + '>-' + XenC.P([x+2,y-2]) + ')'
            inicial = False
        else:
            formulaDI = "(" + formulaDI + "Y" + '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x+1,y-1]) + ')' + '>-' + XenC.P([x+2,y-2]) + '))'

print("--------------------------------------------------FORMULA DI CODIFICADA--------------------------------------------------------")
print("")
print(formulaDI)
print("")
print("--------------------------------------------------FORMULA DI DESCODIFICADA-----------------------------------------------------")
print("")
print(visualizar_formula(formulaDI, XenC))

--------------------------------------------------FORMULA DI CODIFICADA--------------------------------------------------------

((((((((((((((((((((((((((ĎYĈ)>-Ă)Y((ĕYď)>-ĉ))Y((ĜYĖ)>-Đ))Y((ģYĝ)>-ė))Y((ĪYĤ)>-Ğ))Y((ďYĉ)>-ă))Y((ĖYĐ)>-Ċ))Y((ĝYė)>-đ))Y((ĤYĞ)>-Ę))Y((īYĥ)>-ğ))Y((ĐYĊ)>-Ą))Y((ėYđ)>-ċ))Y((ĞYĘ)>-Ē))Y((ĥYğ)>-ę))Y((ĬYĦ)>-Ġ))Y((đYċ)>-ą))Y((ĘYĒ)>-Č))Y((ğYę)>-ē))Y((ĦYĠ)>-Ě))Y((ĭYħ)>-ġ))Y((ĒYČ)>-Ć))Y((ęYē)>-č))Y((ĠYĚ)>-Ĕ))Y((ħYġ)>-ě))Y((ĮYĨ)>-Ģ))

--------------------------------------------------FORMULA DI DESCODIFICADA-----------------------------------------------------

((((((((((((((((((((((((((Hay una X en la casilla (0,2) Y Hay una X en la casilla (1,1)) >  no Hay una X en la casilla (2,0)) Y ((Hay una X en la casilla (0,3) Y Hay una X en la casilla (1,2)) >  no Hay una X en la casilla (2,1))) Y ((Hay una X en la casilla (0,4) Y Hay una X en la casilla (1,3)) >  no Hay una X en la casilla (2,2))) Y ((Hay una X en la casilla (0,5) Y Hay una X en la casilla (1,4)) >  no Hay una X

---

### Fórmula general

Para satisfacer completamente la restricción, se deben unir las 4 fórmulas anteriores de la siguiente manera:

$$\normalsize \bigwedge_{x \in Filas\atop { y \in\{0, 4\}}}\biggl(XenC_{x, y}\wedge XenC_{x, y+1}\to\neg\left(XenC_{x, y+2}\right)\biggr) \wedge \bigwedge_{x \in\{0, 4\}\atop { y \in Columnas}}\biggl(XenC_{x, y}\wedge XenC_{x+1, y}\to\neg\left(XenC_{x+2, y}\right)\biggr) \wedge \bigwedge_{x \in\{0, 4\}\atop { y \in\{0, 4\}}}\biggl(XenC_{x, y}\wedge XenC_{x+1, y+1}\to\neg\left(XenC_{x+2, y+2}\right)\biggr) \wedge \bigwedge_{x \in\{0, 4\}\atop { y \in\{2, 6\}}}\biggl(XenC_{x, y}\wedge XenC_{x+1, y-1}\to\neg\left(XenC_{x+2, y-2}\right)\biggr)$$

Esta fórmula se implementa en Python de la siguiente manera:

In [23]:
Direcciones = []
Direcciones.append(formulaH)
Direcciones.append(formulaV)
Direcciones.append(formulaDD)
Direcciones.append(formulaDI)

Regla2 = Ytoria(Direcciones)

print("--------------------------------------------------REGLA 2 CODIFICADA---------------------------------------------------------")
print("")
print(Regla2)
print("")
print("--------------------------------------------------REGLA 2 DESCODIFICADA------------------------------------------------------")
print("")
print(visualizar_formula(Regla2, XenC))

--------------------------------------------------REGLA 2 CODIFICADA---------------------------------------------------------

(((((((((((((((((((((((((((((((((((((((ĀYć)>-Ď)Y((ćYĎ)>-ĕ))Y((ĎYĕ)>-Ĝ))Y((ĕYĜ)>-ģ))Y((ĜYģ)>-Ī))Y((āYĈ)>-ď))Y((ĈYď)>-Ė))Y((ďYĖ)>-ĝ))Y((ĖYĝ)>-Ĥ))Y((ĝYĤ)>-ī))Y((ĂYĉ)>-Đ))Y((ĉYĐ)>-ė))Y((ĐYė)>-Ğ))Y((ėYĞ)>-ĥ))Y((ĞYĥ)>-Ĭ))Y((ăYĊ)>-đ))Y((ĊYđ)>-Ę))Y((đYĘ)>-ğ))Y((ĘYğ)>-Ħ))Y((ğYĦ)>-ĭ))Y((ĄYċ)>-Ē))Y((ċYĒ)>-ę))Y((ĒYę)>-Ġ))Y((ęYĠ)>-ħ))Y((ĠYħ)>-Į))Y((ąYČ)>-ē))Y((ČYē)>-Ě))Y((ēYĚ)>-ġ))Y((ĚYġ)>-Ĩ))Y((ġYĨ)>-į))Y((ĆYč)>-Ĕ))Y((čYĔ)>-ě))Y((ĔYě)>-Ģ))Y((ěYĢ)>-ĩ))Y((ĢYĩ)>-İ))Y((((((((((((((((((((((((((((((((((((ĀYā)>-Ă)Y((āYĂ)>-ă))Y((ĂYă)>-Ą))Y((ăYĄ)>-ą))Y((ĄYą)>-Ć))Y((ćYĈ)>-ĉ))Y((ĈYĉ)>-Ċ))Y((ĉYĊ)>-ċ))Y((ĊYċ)>-Č))Y((ċYČ)>-č))Y((ĎYď)>-Đ))Y((ďYĐ)>-đ))Y((ĐYđ)>-Ē))Y((đYĒ)>-ē))Y((ĒYē)>-Ĕ))Y((ĕYĖ)>-ė))Y((ĖYė)>-Ę))Y((ėYĘ)>-ę))Y((ĘYę)>-Ě))Y((ęYĚ)>-ě))Y((ĜYĝ)>-Ğ))Y((ĝYĞ)>-ğ))Y((ĞYğ)>-Ġ))Y((ğYĠ)>-ġ))Y((ĠYġ)>-Ģ))Y((ģYĤ)>-ĥ))Y((ĤYĥ)>-Ħ))Y((ĥYĦ)>-ħ))Y((ĦYħ)>-Ĩ))Y((ħYĨ)>-ĩ))Y((ĪYī)>-Ĭ))Y((īYĬ)>-ĭ))Y

---

## Restricción 3

La tercera restricción indica que deben haber como mínimo 3 $X$ en cada fila y columna del tablero. Para satisfacer esta restricción, se debe implementar la misma en filas y columnas.


### Filas ($\rightarrow$)

La fórmula para representar deben haber mínimo 3 $X$ en cada fila es la siguiente:

$$\large \bigwedge_{x \in Filas}\biggl(\bigvee_{y\in Columnas}\biggl(\bigvee_{w\in Columnas \atop{w \neq y}}\biggl(\bigvee_{z\in Columnas \atop{z \neq y \atop{z \neq w}}} \biggl(XenC_{x, y}\wedge XenC_{x, w}\wedge XenC_{x, z}\biggr)\biggr)\biggr)\biggr)$$

Esta fórmula se implementa en Python de la siguiente manera:

In [24]:
formulaF = ''
inicial2 = True

for x in X:
    inicial = True
    formulaF_1 = ''
    for y in Y:
        for w in Y:
            if w!=y:
                for z in Y:
                    if z!=y and z!=w: 
                        if inicial:
                            formulaF_1 = '((' + XenC.P([x,y]) + 'Y' + XenC.P([x,w]) + ')' + 'Y' + XenC.P([x,z]) + ')'
                            inicial = False
                        else:
                            formulaF_1 = "(" + formulaF_1 + "O" + '((' + XenC.P([x,y]) + 'Y' + XenC.P([x,w]) + ')' + 'Y' + XenC.P([x,z]) + ')' + ")"
    if inicial2:
            formulaF = formulaF_1
            inicial2 = False
    else:
        formulaF = "(" + formulaF + 'Y' + formulaF_1 + ')'  

print("--------------------------------------------------FORMULA F CODIFICADA-------------------------------------------------------")
print("")
print(formulaF)
print("")
print("--------------------------------------------------FORMULA F  DESCODIFICADA---------------------------------------------------")
print("")
print(visualizar_formula(formulaF, XenC))

--------------------------------------------------FORMULA F CODIFICADA-------------------------------------------------------

(((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((ĀYć)YĎ)O((ĀYć)Yĕ))O((ĀYć)YĜ))O((ĀYć)Yģ))O((ĀYć)YĪ))O((ĀYĎ)Yć))O((ĀYĎ)Yĕ))O((ĀYĎ)YĜ))O((ĀYĎ)Yģ))O((ĀYĎ)YĪ))O((ĀYĕ)Yć))O((ĀYĕ)YĎ))O((ĀYĕ)YĜ))O((ĀYĕ)Yģ))O((ĀYĕ)YĪ))O((ĀYĜ)Yć))O((ĀYĜ)YĎ))O((ĀYĜ)Yĕ))O((ĀYĜ)Yģ))O((ĀYĜ)YĪ))O((ĀYģ)Yć))O((ĀYģ)YĎ))O((ĀYģ)Yĕ))O((ĀYģ)YĜ))O((ĀYģ)YĪ))O((ĀYĪ)Yć))O((ĀYĪ)YĎ))O((ĀYĪ)Yĕ))O((ĀYĪ)YĜ))O((ĀYĪ)Yģ))O((ćYĀ)YĎ))O((ćYĀ)Yĕ))O((ćYĀ)YĜ))O((ćYĀ)Yģ))O((ćYĀ)YĪ))O((ćYĎ)YĀ))O((ćYĎ)Yĕ))O((ćYĎ)YĜ))O((ćYĎ)Yģ))O((ćYĎ)YĪ))O((ćYĕ)YĀ))O((ćYĕ)YĎ))O((ćYĕ)YĜ))O((ćYĕ)Yģ))O((ćYĕ)YĪ))O((ćYĜ)YĀ))O((ćYĜ)YĎ))O((ćYĜ)Yĕ))O((ćYĜ)Yģ))O((ćYĜ)YĪ))O((ćYģ)YĀ))O((ćYģ)YĎ))O((ćYģ)Yĕ))O((ćYģ)YĜ))O((ćYģ)YĪ))O((ćYĪ)YĀ))O((ćYĪ)YĎ))O((ćYĪ)Yĕ))O((ćYĪ)YĜ))O((ćYĪ)Yģ))

### Columnas ($\downarrow$)

La fórmula para representar deben haber mínimo 3 $X$ en cada colunma es la siguiente:

$$\large \bigwedge_{y \in Columnas}\biggl(\bigvee_{x\in Filas}\biggl(\bigvee_{w\in Filas \atop{w \neq x}}\biggl(\bigvee_{z\in Filas \atop{z \neq x \atop{z \neq w}}} \biggl(XenC_{x, y}\wedge XenC_{w, y}\wedge XenC_{z, y}\biggr)\biggr)\biggr)\biggr)$$

Esta fórmula se implementa en Python de la siguiente manera:

In [28]:
formulaC = ''
inicial2 = True

for y in Y:
    inicial = True
    formulaC_1 = ''
    for x in X:
        for w in X:
            if w!=x:
                for z in X:
                    if z!=x and z!=w: 
                        if inicial:
                            formulaC_1 = '((' + XenC.P([x,y]) + 'Y' + XenC.P([w,y]) + ')' + 'Y' + XenC.P([z,y]) + ')'
                            inicial = False
                        else:
                            formulaC_1 = "(" + formulaC_1 + "O" + '((' + XenC.P([x,y]) + 'Y' + XenC.P([w,y]) + ')' + 'Y' + XenC.P([z,y]) + ')' + ")"
    if inicial2:
        formulaC = formulaC_1
        inicial2 = False
    else:
        formulaC = "(" + formulaC + 'Y' + formulaC_1 + ')'

print("--------------------------------------------------FORMULA C CODIFICADA-------------------------------------------------------")
print("")
print(formulaC)
print("")
print("--------------------------------------------------FORMULA C DESCODIFICADA---------------------------------------------------")
print("")
print(visualizar_formula(formulaC, XenC))

--------------------------------------------------FORMULA C CODIFICADA-------------------------------------------------------

(((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((ĀYā)YĂ)O((ĀYā)Yă))O((ĀYā)YĄ))O((ĀYā)Yą))O((ĀYā)YĆ))O((ĀYĂ)Yā))O((ĀYĂ)Yă))O((ĀYĂ)YĄ))O((ĀYĂ)Yą))O((ĀYĂ)YĆ))O((ĀYă)Yā))O((ĀYă)YĂ))O((ĀYă)YĄ))O((ĀYă)Yą))O((ĀYă)YĆ))O((ĀYĄ)Yā))O((ĀYĄ)YĂ))O((ĀYĄ)Yă))O((ĀYĄ)Yą))O((ĀYĄ)YĆ))O((ĀYą)Yā))O((ĀYą)YĂ))O((ĀYą)Yă))O((ĀYą)YĄ))O((ĀYą)YĆ))O((ĀYĆ)Yā))O((ĀYĆ)YĂ))O((ĀYĆ)Yă))O((ĀYĆ)YĄ))O((ĀYĆ)Yą))O((āYĀ)YĂ))O((āYĀ)Yă))O((āYĀ)YĄ))O((āYĀ)Yą))O((āYĀ)YĆ))O((āYĂ)YĀ))O((āYĂ)Yă))O((āYĂ)YĄ))O((āYĂ)Yą))O((āYĂ)YĆ))O((āYă)YĀ))O((āYă)YĂ))O((āYă)YĄ))O((āYă)Yą))O((āYă)YĆ))O((āYĄ)YĀ))O((āYĄ)YĂ))O((āYĄ)Yă))O((āYĄ)Yą))O((āYĄ)YĆ))O((āYą)YĀ))O((āYą)YĂ))O((āYą)Yă))O((āYą)YĄ))O((āYą)YĆ))O((āYĆ)YĀ))O((āYĆ)YĂ))O((āYĆ)Yă))O((āYĆ)YĄ))O((āYĆ)Yą))

---

### Fórmula general

Para satisfacer completamente la restricción, se deben unir las 2 fórmulas anteriores de la siguiente manera:

$$\large \bigwedge_{x \in Filas}\biggl(\bigvee_{y\in Columnas}\biggl(\bigvee_{w\in Columnas \atop{w \neq y}}\biggl(\bigvee_{z\in Columnas \atop{z \neq y \atop{z \neq w}}} \biggl(XenC_{x, y}\wedge XenC_{x, w}\wedge XenC_{x, z}\biggr)\biggr)\biggr)\biggr)\wedge \bigwedge_{y \in Columnas}\biggl(\bigvee_{x\in Filas}\biggl(\bigvee_{w\in Filas \atop{w \neq x}}\biggl(\bigvee_{z\in Filas \atop{z \neq x \atop{z \neq w}}} \biggl(XenC_{x, y}\wedge XenC_{w, y}\wedge XenC_{z, y}\biggr)\biggr)\biggr)\biggr)$$

Esta fórmula se implementa en Python de la siguiente manera:

In [26]:
Direcciones2 = []
Direcciones2.append(formulaF)
Direcciones2.append(formulaC)
Regla3 = Ytoria(Direcciones2)

print("--------------------------------------------------REGLA 3 CODIFICADA---------------------------------------------------------")
print("")
print(Regla3)
print("")
print("--------------------------------------------------REGLA 3 DESCODIFICADA------------------------------------------------------")
print("")
print(visualizar_formula(Regla3, XenC))

--------------------------------------------------REGLA 3 CODIFICADA---------------------------------------------------------

((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((ĀYć)YĎ)O((ĀYć)Yĕ))O((ĀYć)YĜ))O((ĀYć)Yģ))O((ĀYć)YĪ))O((ĀYĎ)Yć))O((ĀYĎ)Yĕ))O((ĀYĎ)YĜ))O((ĀYĎ)Yģ))O((ĀYĎ)YĪ))O((ĀYĕ)Yć))O((ĀYĕ)YĎ))O((ĀYĕ)YĜ))O((ĀYĕ)Yģ))O((ĀYĕ)YĪ))O((ĀYĜ)Yć))O((ĀYĜ)YĎ))O((ĀYĜ)Yĕ))O((ĀYĜ)Yģ))O((ĀYĜ)YĪ))O((ĀYģ)Yć))O((ĀYģ)YĎ))O((ĀYģ)Yĕ))O((ĀYģ)YĜ))O((ĀYģ)YĪ))O((ĀYĪ)Yć))O((ĀYĪ)YĎ))O((ĀYĪ)Yĕ))O((ĀYĪ)YĜ))O((ĀYĪ)Yģ))O((ćYĀ)YĎ))O((ćYĀ)Yĕ))O((ćYĀ)YĜ))O((ćYĀ)Yģ))O((ćYĀ)YĪ))O((ćYĎ)YĀ))O((ćYĎ)Yĕ))O((ćYĎ)YĜ))O((ćYĎ)Yģ))O((ćYĎ)YĪ))O((ćYĕ)YĀ))O((ćYĕ)YĎ))O((ćYĕ)YĜ))O((ćYĕ)Yģ))O((ćYĕ)YĪ))O((ćYĜ)YĀ))O((ćYĜ)YĎ))O((ćYĜ)Yĕ))O((ćYĜ)Yģ))O((ćYĜ)YĪ))O((ćYģ)YĀ))O((ćYģ)YĎ))O((ćYģ)Yĕ))O((ćYģ)YĜ))O((ćYģ)YĪ))O((ćYĪ)YĀ))O((ćYĪ)YĎ))O((ćYĪ)Yĕ))O((ćYĪ)YĜ))O((ćYĪ)Yģ)

---

## Reglas

Ahora se unen las 2 reglas en una sola formula de la siguiente manera:
$$\normalsize\biggl(\bigwedge_{x \in Filas\atop { y \in\{0, 4\}}}\bigl(XenC_{x, y}\wedge XenC_{x, y+1}\to\neg\left(XenC_{x, y+2}\right)\bigr) \wedge \bigwedge_{x \in\{0, 4\}\atop { y \in Columnas}}\bigl(XenC_{x, y}\wedge XenC_{x+1, y}\to\neg\left(XenC_{x+2, y}\right)\bigr) \wedge \bigwedge_{x \in\{0, 4\}\atop { y \in\{0, 4\}}}\bigl(XenC_{x, y}\wedge XenC_{x+1, y+1}\to\neg\left(XenC_{x+2, y+2}\right)\bigr)\wedge \bigwedge_{x \in\{0, 4\}\atop { y \in\{2, 6\}}}\bigl(XenC_{x, y}\wedge XenC_{x+1, y-1}\to\neg\left(XenC_{x+2, y-2}\right)\bigr)\biggr)\wedge\biggl(\bigwedge_{x \in Filas}\bigl(\bigvee_{y\in Columnas}\bigl(\bigvee_{w\in Columnas \atop{w \neq y}}\bigl(\bigvee_{z\in Columnas \atop{z \neq y \atop{z \neq w}}} \bigl(XenC_{x, y}\wedge XenC_{x, w}\wedge XenC_{x, z}\bigr)\bigr)\bigr)\bigr)\wedge \bigwedge_{y \in Columnas}\bigl(\bigvee_{x\in Filas}\bigl(\bigvee_{w\in Filas \atop{w \neq x}}\bigl(\bigvee_{z\in Filas \atop{z \neq x \atop{z \neq w}}} \bigl(XenC_{x, y}\wedge XenC_{w, y}\wedge XenC_{z, y}\bigr)\bigr)\bigr)\bigr)\biggr)$$

Esta fórmula se implementa en Python de la siguiente manera:

In [ ]:
Reglas_ = []
Reglas_.append(Regla2)
Reglas_.append(Regla3)

Reglas = Ytoria(Reglas_)

print("--------------------------------------------------REGLAS CODIFICADA---------------------------------------------------------")
print("")
print(Reglas)
print("")
print("--------------------------------------------------REGLAS DESCODIFICADA------------------------------------------------------")
print("")
print(visualizar_formula(Reglas, XenC))